In [3]:
from dotenv import load_dotenv
from tqdm import tqdm
import os
from openai import OpenAI
import pandas as pd
import sys
import pandas as pd
csv_path = None
load_dotenv()

False

In [ ]:
# Local
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN"),
)

csv_path = '../Data/new_england_indeed_jobs.csv'

In [4]:
# Remote
from google.colab import userdata

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=userdata.get('HF_TOKEN'),
)

csv_path = 'new_england_indeed_jobs.csv'

In [6]:
df = pd.read_csv(csv_path)
df

,id,site,job_url,job_url_direct,title,company,location,date_posted,job_type,salary_source,...,company_addresses,company_num_employees,company_revenue,company_description,skills,experience_range,company_rating,company_reviews_count,vacancy_count,work_from_home_type
0,in-9f0c48dc5551762a,indeed,https://www.indeed.com/viewjob?jk=9f0c48dc5551...,http://www.indeed.com/job/-home-caregiver-9f0c...,In - Home Caregiver,Coastal Care Solutions,"Portland, ME, US",2025-11-08,parttime,NaN,...,NaN,NaN,NaN,NaN,"patient care, first aid, elderly care",NaN,NaN,NaN,NaN,NaN
1,in-16f94c9ca0e56be7,indeed,https://www.indeed.com/viewjob?jk=16f94c9ca0e5...,http://www.indeed.com/job/infanttoddlerprescho...,Infant/Toddler/Preschool Teacher,Pearlite Montessori Home,"South Portland, ME, US",2025-11-08,fulltime,NaN,...,NaN,NaN,NaN,NaN,"Montessori, early childhood, classroom management",NaN,NaN,NaN,NaN,NaN
2,in-948094920b9c3dd6,indeed,https://www.indeed.com/viewjob?jk=948094920b9c...,https://www.careersatmainehealth.org/jobs/1701...,Cardiac Sonographer - Cardiopulmonary,MaineHealth,"Biddeford, ME, US",2025-11-08,fulltime,NaN,...,"110 Free Street\r\nPortland, ME 04102","10,000+",$25M to $100M (USD),The MaineHealth system offers outstanding oppo...,"ultrasound, echocardiography, cardiac imaging",NaN,NaN,NaN,NaN,NaN
3,in-7e5c00a5634552e0,indeed,https://www.indeed.com/viewjob?jk=7e5c00a56345...,https://retailcareers.staples.com//job/-/-/495...,Retail Sales Associate,Staples,"Auburn, ME, US",2025-11-08,parttime,NaN,...,"500 Staples Drive, Framingham, MA 01702","10,000+",more than $10B (USD),"Staples is a world-class retail, online and de...","sales, POS",NaN,NaN,NaN,NaN,NaN
4,in-148df71f298ae14d,indeed,https://www.indeed.com/viewjob?jk=148df71f298a...,http://www.indeed.com/job/strength-trainer-gro...,Strength Trainer (Group Classes),Mome Studios,"South Portland, ME, US",2025-11-08,parttime,direct_data,...,NaN,NaN,NaN,NaN,"strength training,group instruction",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,in-329e2be3c7ef110a,indeed,https://www.indeed.com/viewjob?jk=329e2be3c7ef...,http://www.indeed.com/job/primary-and-urgent-c...,"Primary and Urgent Care Providers (MD, DO, APR...",DestaHealth,"Stamford, CT, US",2025-11-07,fulltime,direct_data,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5396,in-88ed51b41c3ea6f3,indeed,https://www.indeed.com/viewjob?jk=88ed51b41c3e...,http://www.indeed.com/job/customer-service-adv...,Customer Service Advisor,Splash Car Wash,"Greenwich, CT, US",2025-11-07,fulltime,NaN,...,472 Wheelers Farm Road\r\nSuite 304\r\nMilford...,"1,001 to 5,000",NaN,Splash Car Wash is a 71 site car wash and deta...,NaN,NaN,NaN,NaN,NaN,NaN
5397,in-1adb304c2c2a3f6f,indeed,https://www.indeed.com/viewjob?jk=1adb304c2c2a...,https://appsparamedicalct.betterteam.com/mobil...,Mobile Phlebotomist,APPS Paramedical,"New Haven, CT, US",2025-11-07,contract,direct_data,...,"APPS\r\nOne Jericho Plaza\r\nJericho, New York...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5398,in-8e7d1d763b9c1404,indeed,https://www.indeed.com/viewjob?jk=8e7d1d763b9c...,https://td.wd3.myworkdayjobs.com/en-US/TD_Bank...,"Retail Banker I - Wethersfield, CT",TD Bank,"Wethersfield, CT, US",2025-11-07,fulltime,direct_data,...,"Toronto, ON","10,000+",more than $10B (USD),TD is one of the world’s leading global financ...,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import json

def get_skills(json_string):
    completion = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=[
            {
                "role": "developer",
                "content": """You are a skill extraction model.
                You will be provided multiple job descriptions separated like json array entries.
                Extract 1-3 relevant HARD skills in a simple comma-separated list with each skill being 1-2 words.
                Do this for each given job description, and combine them into a json-like structure of array of strings similar to the input.
                The output should be able to parsed by a json parser, so do not provide any extra text.
                Do NOT extract soft skills (e.g., \"communication,\" \"scheduling,\" etc. are examples of soft skills and not hard skills)"""
            },
            {
                "role": "user",
                "content": json_string
            }
        ],
    )
    val = completion.choices[0].message.content
    return val

In [8]:
start = 85

In [ ]:
from time import sleep

# For loop folding
wait_time = 60*3
batch_size = 4
for i in tqdm(range(start, len(df) // batch_size)):
    try:
        range_generator = range(i * batch_size, min((i + 1) * batch_size, len(df)))
        rows = [df.iloc[j] for j in range_generator]
        json_string = json.dumps([f"\n{i+1}. {row}" for i, row in enumerate(rows)])

        while True:
            try:
                result = get_skills(json_string)
                parsed_json_dict = json.loads(result)
                element_index = 0
                for j in range_generator:
                    df.at[j, "skills"] = parsed_json_dict[element_index]
                    element_index += 1
                break

            except Exception as e:
                print(f"\nRate limit at batch {i} with size {batch_size}: {e}\nWaiting {wait_time} secs before continuing...", file=sys.stderr)
                sys.stderr.flush()
                df.to_csv(csv_path, index=False)
                sleep(wait_time)
    except KeyboardInterrupt:
        start = i

  0%|          | 0/1265 [00:00<?, ?it/s]
Rate limit at row 340: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}
Waiting 180 secs before continuing...
  0%|          | 3/1265 [03:04<14:35:09, 41.61s/it]
Rate limit at row 352: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}
Waiting 180 secs before continuing...
  0%|          | 5/1265 [06:08<21:53:53, 62.57s/it]
Rate limit at row 360: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}
Waiting 180 secs before continuing...
  1%|          | 7/1265 [09:11<24:26:12, 69.93s/it] 
Rate limit at row 368: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subsc